In [4]:
import nfl_data_py as nfl
import pandas as pd
pd.options.display.max_columns = None
data = nfl.import_pbp_data([2020], downcast=True, cache=False, alt_path=None)
data.head()


2020 done.
Downcasting floats.


,play_id,game_id,old_game_id_x,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,td_player_name,td_player_id,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,vegas_wpa,vegas_home_wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,air_wpa,yac_wpa,comp_air_wpa,comp_yac_wpa,total_home_comp_air_wpa,total_away_comp_air_wpa,total_home_comp_yac_wpa,total_away_comp_yac_wpa,total_home_raw_air_wpa,total_away_raw_air_wpa,total_home_raw_yac_wpa,total_away_raw_yac_wpa,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,touchback,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,own_kickoff_recovery,own_kickoff_recovery_td,qb_hit,rush_attempt,pass_attempt,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,kickoff_attempt,punt_attempt,fumble,complete_pass,assist_tackle,lateral_reception,lateral_rush,lateral_return,lateral_recovery,passer_player_id,passer_player_name,passing_yards,receiver_player_id,receiver_player_name,receiving_yards,rusher_player_id,rusher_player_name,rushing_yards,lateral_receiver_player_id,lateral_receiver_player_name,lateral_receiving_yards,lateral_rusher_player_id,lateral_rusher_player_name,lateral_rushing_yards,lateral_sack_player_id,lateral_sack_player_name,interception_player_id,interception_player_name,lateral_interception_player_id,lateral_interception_player_name,punt_returner_player_id,punt_returner_player_name,lateral_punt_returner_player_id,lateral_punt_returner_player_name,kickoff_returner_player_name,kickoff_returner_player_id,lateral_kickoff_returner_player_id,lateral_kickoff_returner_player_name,punter_player_id,punter_player_name,kicker_player_name,kicker_player_id,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,blocked_player_id,blocked_player_name,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,tackle_for_loss_2_player_id,tackle_for_loss_2_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_team,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_2_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,solo_tackle_1_team,solo_tackle_2_team,solo_tackle_1_player_id,solo_tackle_2_player_id,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_ta

In [ ]:
cols = ['play_id', 'game_id', 'home_team', 'away_team', 'posteam', 
        'yardline_100', 'ydstogo', 'half_seconds_remaining', 'game_half', 
        'drive', 'down', 'qb_dropback', 'qb_kneel', 'qb_spike',
        'posteam_timeouts_remaining', 'defteam_timeouts_remaining', 
        'score_differential', 'epa', 'home_wp', 'away_wp']

df_train = data[cols]
df_train.sort_values(by=['game_id', 'drive'])
df_train.to_csv('')

,play_id,game_id,home_team,away_team,posteam,yardline_100,ydstogo,half_seconds_remaining,game_half,drive,down,qb_dropback,qb_kneel,qb_spike,posteam_timeouts_remaining,defteam_timeouts_remaining,score_differential,epa,home_wp,away_wp
0,1.0,2020_01_ARI_SF,SF,ARI,None,NaN,0.0,1800.0,Half1,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.000000,0.546262,0.453738
1,39.0,2020_01_ARI_SF,SF,ARI,SF,35.0,0.0,1800.0,Half1,1.0,NaN,0.0,0.0,0.0,3.0,3.0,0.0,0.000000,0.546262,0.453738
2,54.0,2020_01_ARI_SF,SF,ARI,SF,75.0,10.0,1800.0,Half1,1.0,1.0,1.0,0.0,0.0,3.0,3.0,0.0,1.294838,0.546262,0.453738
3,93.0,2020_01_ARI_SF,SF,ARI,SF,55.0,10.0,1782.0,Half1,1.0,1.0,0.0,0.0,0.0,3.0,3.0,0.0,0.857214,0.579976,0.420024
4,118.0,2020_01_ARI_SF,SF,ARI,SF,41.0,10.0,1739.0,Half1,1.0,1.0,0.0,0.0,0.0,3.0,3.0,0.0,-0.454665,0.604475,0.395525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47700,4280.0,2020_21_KC_TB,TB,KC,KC,10.0,6.0,100.0,Half2,21.0,2.0,1.0,0.0,0.0,0.0,3.0,-22.0,-4.266047,0.998739,0.001261
47701,4307.0,2020_21_KC_TB,TB,KC,TB,80.0,10.0,93.0,Half2,22.0,1.0,0.0,1.0,0.0,3.0,0.0,22.0,0.000000,0.999959,0.000041
47702,4328.0,2020_21_KC_TB,TB,KC,TB,81.0,11.0,50.0,Half2,22.0,2.0,0.0,1.0,0.0,3.0,0.0,22.0,0.000000,0.999946,0.000054
47703,4349.0,2020_21_KC_TB,TB,KC,TB,81.0,11.0,30.0,Half2,22.0,3.0,0.0,1.0,0.0,3.0,0.0,22.0,0.000000,0.999933,0.000067
